# Analysing Fit

In [14]:
import numpy as np
import matplotlib.pyplot as plt
from data_load import load_data, pmumu, nll, lam_i, parabolic, nll_dm2, nll_theta

data, flux = load_data()

For this code, I will use the values given by the parabolic minimiser. 

For the NLL:

$$
\Delta NLL = 1 \text{ (absolute units)} =\pm 1\sigma
$$

Since the NLL equation has dimensionless terms they can be refered to as absolute units

If we use Taylor expansion on the NLL around the min for one parameter, say $\Delta m^2$, we obtain:

$$
NLL(\Delta m^2) \approx NLL_{min} + \frac{1}{2}(\Delta m_2 - \Delta m^2_{min})^2NLL_{min}
$$

Therefore:

$$
\Delta NLL(\Delta m^2) \equiv NLL(\Delta m^2) - NLL_{min} \approx\frac{1}{2}(\Delta m_2 - \Delta m^2_{min})^2NLL_{min}
$$

We can see that near the minimum the NLL, and any relatively smooth function also, will behave like a parabola (hence why we can and did use a parabolic minimiser)

By using maximum liklihood theory (minimum is approx Gaussian around the true value) variance and Fisher information (and maths that I will skip here because I am lazy), we can obtain the curvature-variance relation:

$$
\sigma^2 \approx \frac{2}{NLL_{min}}
$$

We can substitute this relation into our Taylor Expansion and then achieve:

$$
\Delta NLL(\Delta m^2) \approx \frac{(\Delta m^2 -\Delta m^2_{min})^2}{\sigma^2}
$$

By setting $\Delta NLL = 1$ we can then obtain $|\Delta m^2 - \Delta m^2_{min}| = \sigma$ and we can see that values of the parameter, in this case $\Delta m^2$ where the NLL is 1 larger than its min are $\Delta m^2 \approx \Delta m^2_{min} \pm \sigma$

In [ ]:
# rerunning parabolic minimum calcs
dm2_0 = 1.5e-3
dm2_1 = 2.4e-3
dm2_2 = 3.5e-3
dm2_min, nll_dm2_min = parabolic(nll_dm2, dm2_0, dm2_1, dm2_2, tol = 1e-5)

print(dm2_min, nll_dm2_min)

t0 = 0.4
t1 = 0.7
t2 = 1.0

theta_min, nll_theta_min = parabolic(nll_theta, t0, t1, t2, tol = 1e-3)

0.0024473468435815774 -73.64525252115868


In [ ]:


def _curvature_sigma(f, theta_min, h=1e-3):
    """
    Estimate the 1σ error from the local curvature of the NLL around theta_min,
    using a central finite difference to approximate the second derivative.

    Near the minimum:
        NLL(theta) ≈ NLL_min + 0.5 * NLL''(theta_min) * (theta - theta_min)^2
    Setting ΔNLL = 1 gives:
        1 ≈ 0.5 * NLL'' * σ^2  ->  σ ≈ sqrt(2 / NLL'')
    """
    f0 = f(theta_min)
    fp = f(theta_min + h)
    fm = f(theta_min - h)

    second_deriv = (fp - 2.0*f0 + fm) / (h**2)

    if second_deriv <= 0:
        # Shouldn't really happen for a proper minimum, but guard anyway.
        raise RuntimeError("Non-positive curvature at minimum; cannot estimate sigma from curvature.")

    sigma = np.sqrt(2.0 / second_deriv)
    return sigma


def _find_one_side_deltaNLL(f, theta_min, f_min,
                            direction=+1,
                            delta=1.0,
                            initial_step=1e-3,
                            tol=1e-4,
                            max_iter=60):
    """
    Find where NLL(theta) - NLL_min = delta on one side of theta_min.

    direction = +1 : search towards larger theta
    direction = -1 : search towards smaller theta

    Strategy:
      1. Step out from theta_min until ΔNLL crosses 'delta'.
      2. Use bisection between the two points to solve ΔNLL = delta.
    """
    # Initial step (signed)
    step = direction * abs(initial_step)

    x_lo = theta_min
    f_lo = f_min

    x_hi = theta_min + step
    f_hi = f(x_hi)

    # Expand the step until we bracket ΔNLL = delta
    n = 0
    while (f_hi - f_min) < delta and n < max_iter:
        x_lo, f_lo = x_hi, f_hi
        step *= 2.0  # grow step size to speed up
        x_hi = theta_min + step
        f_hi = f(x_hi)
        n += 1

    if (f_hi - f_min) < delta:
        raise RuntimeError("Could not bracket ΔNLL = {} on direction {}."
                           .format(delta, direction))

    # Now we have x_lo with ΔNLL < delta and x_hi with ΔNLL >= delta.
    for _ in range(max_iter):
        x_mid = 0.5 * (x_lo + x_hi)
        f_mid = f(x_mid)
        d_mid = f_mid - f_min

        if abs(x_hi - x_lo) < tol:
            return x_mid

        if d_mid < delta:
            x_lo, f_lo = x_mid, f_mid
        else:
            x_hi, f_hi = x_mid, f_mid

    # If we reach here, just return the last midpoint
    return x_mid


def estimate_theta_error(f, theta_min, f_min,
                         delta=1.0,
                         initial_step=1e-3,
                         tol=1e-4):
    """
    Fully automatic error estimation for θ_23 from the NLL curve.

    Inputs
    ------
    f         : function handle, NLL(θ)
    theta_min : best–fit θ (output from parabolic)
    f_min     : NLL(θ_min)
    delta     : ΔNLL defining 1σ (default = 1)
    initial_step : starting step size for the scan
    tol       : tolerance for the bisection in θ

    Returns
    -------
    A dict with:
      - 'theta_min'          : best–fit θ
      - 'sigma_scan_minus'   : 1σ error from ΔNLL = +delta on low side
      - 'sigma_scan_plus'    : 1σ error from ΔNLL = +delta on high side
      - 'sigma_curvature'    : 1σ error from curvature at the minimum
    """
    # Method 1: scan for ΔNLL = 1 in both directions
    theta_plus = _find_one_side_deltaNLL(
        f, theta_min, f_min,
        direction=+1,
        delta=delta,
        initial_step=initial_step,
        tol=tol
    )

    theta_minus = _find_one_side_deltaNLL(
        f, theta_min, f_min,
        direction=-1,
        delta=delta,
        initial_step=initial_step,
        tol=tol
    )

    sigma_plus = theta_plus - theta_min       # should be > 0
    sigma_minus = theta_min - theta_minus     # should be > 0

    # Method 2: curvature-based estimate
    sigma_curv = _curvature_sigma(f, theta_min, h=initial_step)

    return {
        "theta_min": theta_min,
        "sigma_scan_minus": sigma_minus,
        "sigma_scan_plus": sigma_plus,
        "sigma_curvature": sigma_curv,
    }